# Check for updates

**Current version : 2024.01.25**

In [ ]:
import ipywidgets as widgets
from IPython.utils import capture
with capture.capture_output() as cap: 
    !pip install -q -U requests
import requests

button = widgets.Button(description="Download now", button_style='success')
output = widgets.Output()

def on_button_clicked(b, url, version):
  with output:
    print('Downloading...')
    !wget -nv -O /workspace/sd_webui_runpod_{version}.ipynb {url}
    print(f'sdxl_webui_runpod_{version}.ipynb downloaded in the root directory')

currentVersion = '2024.01.25'
updateURL = 'https://raw.githubusercontent.com/ffxvs/sd-webui-complete-setup/main/update.json'
response = requests.get(updateURL)

if response.status_code == 200:
    res = response.json()
    for data in res['data']:
        if data['id'] == 'runpod':
            for variant in data['variants']:
                if variant['type'] == 'sdxl':
                    print('Current version : ' + currentVersion)
                    print('Latest version  : ' + variant['version'])
                    if (variant['version'] > currentVersion):
                        print('\nThere is new version')
                        button.on_click(lambda b: on_button_clicked(b, variant['url'], variant['version']))
                        display(button, output)
                        print('\nChangelog :')
                        print(variant['changelog'])
                    else:
                        print('\nThis is the latest version')                
else:
    print('Failed to check for updates') 
    print('Response code :', response.status_code)

# 1. Requirements

### 1.1. Function and Data
**REQUIRED EVERY TIME YOU RELOAD**

In [ ]:
# Function and Data (REQUIRED)

import os, re
import ipywidgets as widgets
from IPython.utils import capture
from IPython.display import clear_output

########################### GLOBAL PATHS AND FUNCTION ###########################

boolean = [False, True]

# Paths
root = '/workspace'
webui = root + '/stable-diffusion-webui'
modulesPath = webui + '/modules'
outputsPath = webui + '/outputs'
extensionsPath = webui + '/extensions'
controlNetModelsPath = extensionsPath + "/sd-webui-controlnet/models"
embeddingsPath = webui + "/embeddings"
modelsPath = webui + "/models/Stable-diffusion"
loraPath = webui + "/models/Lora"
upscalerPath = webui + "/models/ESRGAN"
vaePath = webui + "/models/VAE"
dependenciesPath = root + '/dependencies'


# Create symlink
def symlink(source, destination):
    if os.path.exists(source) and not os.path.islink(destination):
        !rm -r -f {destination}
        !ln -s {source} {destination}

# Complete message
def inf(msg, style, width): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=width));display(inf)

# Model Mapper
def modelMapper(name, version, url):
    return {
        'name': name,
        'version': version,
        'url': url
    }

# Selected Models
def selectedModels(models):
    modelList = []
    for model in models:
        isSelected = eval(model['id'])
        if isSelected != 'Select version...':
            for variant in model['variants']:
                if variant['version'] == isSelected:
                    modelList.append(modelMapper(
                        model['name'],
                        variant['version'],
                        variant['url']
                    ))
    return modelList

# Aria2c
def downloader(url, path, overwrite=False):
    args = '--download-result=hide --console-log-level=error -c -x 16 -s 16 -k 1M '
    if overwrite: args += '--allow-overwrite'
    formattedURL = '"' + url + '"'
    if bool(re.search('\/[\w\.-]+\.\w+$', url)):
        filename = url.split('/')[-1]
        !aria2c {args} {formattedURL} -d {path} -o {filename}
    else:
        !aria2c {args} {formattedURL} -d {path}

# Git Clone
def silentClone(command, path, update=False, overwrite=False):
    directory = command.split('/')[-1]
    if os.path.exists(path + '/' + directory):
        if update:
            %cd {path}/{directory}
            !git pull -q
        elif overwrite:
            !rm -r {path}/{directory}
            !git clone -q --depth 10 {command} {path}/{directory}
    else:
        !git clone -q --depth 10 {command} {path}/{directory}
        
# WGet
def silentGet(command):
    !wget -nv {command}

# Resources downloader
def defaultResources(resourceList, resourcePath):
    for resource in resourceList:
        print('\n' + resource['name'] + '...')
        downloader(resource['url'], resourcePath)

def selectedResources(resourceList, resourcePath):
    for resource in resourceList:
        if eval(resource['id']):
            print('\n' + resource['name'] + '...')
            for url in resource['url']:
                downloader(url, resourcePath)
                print('')
                
def otherResources(resourceList, resourcePath):
    for resource in resourceList:
        print('\n' + resource)
        downloader(resource, resourcePath)

# Remove git from directory
def removeGit(directory):
    for (folder, subfolders, files) in os.walk(directory):
        if subfolders:
            for subfolder in subfolders:
                if subfolder == '.git':
                    pathFolder = os.path.join(folder, subfolder)
                    !rm -r {pathFolder}

        for file in files:
            if file == '.gitattributes' or file == 'readme.md' or file == 'README.md':
                pathFile = os.path.join(folder, file)
                !rm {pathFile}


################################ DATA OBJECT ################################
################################ CONTROLNET ################################

controlNetURLs = {
    'blur': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur.safetensors'],
    'blur_anime': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur_anime.safetensors'],
    'canny': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_mid.safetensors'],
    'canny_anime': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_canny_anime.safetensors'],
    'depth': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_mid.safetensors'],
    'depth_anime': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_depth_anime.safetensors'],
    'ipadapter': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_xl.pth'],
    'lineart': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors'],
    'openpose': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose_256lora.safetensors'],
    'openpose_anime': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_openpose_anime_v2.safetensors'],
    'recolor': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_128lora.safetensors'],
    'scribble_anime': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_scribble_anime.safetensors'],
    'sketch': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors'],
    'softedge': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_softedge.safetensors']
}

################################ EXTENSIONS ################################

defaultExtensions = [
    {
        'name': 'Aspect Ratio Helper',
        'url': 'https://github.com/thomasasfk/sd-webui-aspect-ratio-helper',
    },
    {
        'name': 'Canvas Zoom',
        'url': 'https://github.com/richrobber2/canvas-zoom',
    },
    {
        'name': 'Cleaner',
        'url': 'https://github.com/novitalabs/sd-webui-cleaner',
    },
    {
        'name': 'Controlnet',
        'url': 'https://github.com/Mikubill/sd-webui-controlnet',
    },
    {
        'name': 'SD Delete Button',
        'url': 'https://github.com/reforget-id/sd_delete_button',
    },
    {
        'name': 'SD Model Downloader',
        'url': 'https://github.com/Iyashinouta/sd-model-downloader',
    },
    {
        'name': 'State',
        'url': 'https://github.com/ilian6806/stable-diffusion-webui-state',
    },
    {
        'name': 'Ultimate SD Upscale',
        'url': 'https://github.com/Coyote-A/ultimate-upscale-for-automatic1111',
    }
]

extensionList = [
    {
        'id': 'adetailer',
        'name': 'Adetailer (After Detailer)',
        'url': 'https://github.com/Bing-su/adetailer',
    },
    {
        'id': 'animateDiff',
        'name': 'AnimateDiff',
        'url': 'https://github.com/continue-revolution/sd-webui-animatediff',
    },
    {
        'id': 'bmab',
        'name': 'BMAB',
        'url': 'https://github.com/portu-sim/sd-webui-bmab'
    },
    {
        'id': 'depthLib',
        'name': 'Depth Map Library',
        'url': 'https://github.com/wywywywy/sd-webui-depth-lib',
    },
    {
        'id': 'huggingFace',
        'name': 'Hugging Face',
        'url': 'https://github.com/camenduru/stable-diffusion-webui-huggingface',
    },
    {
        'id': 'infiniteImg',
        'name': 'Infinite Image Browsing',
        'url': 'https://github.com/zanllp/sd-webui-infinite-image-browsing',
    },
    {
        'id': 'inpaintAny',
        'name': 'Inpaint Anything',
        'url': 'https://github.com/Uminosachi/sd-webui-inpaint-anything',
    },
    {
        'id': 'latentCouple',
        'name': 'Latent Couple',
        'url': 'https://github.com/aria1th/stable-diffusion-webui-two-shot',
    },
    {
        'id': 'miniPaint',
        'name': 'Mini Paint',
        'url': 'https://github.com/0Tick/a1111-mini-paint',
    },
    {
        'id': 'npw',
        'name': 'Negative Prompt Weight',
        'url': 'https://github.com/muerrilla/stable-diffusion-NPW',
    },
    {
        'id': 'openOutpaint',
        'name': 'openOutpaint',
        'url': 'https://github.com/zero01101/openOutpaint-webUI-extension',
    },
    {
        'id': 'photopea',
        'name': 'Photopea',
        'url': 'https://github.com/yankooliveira/sd-webui-photopea-embed',
    },
    {
        'id': 'promptHistory',
        'name': 'Prompt History',
        'url': 'https://github.com/namkazt/sd-webui-prompt-history',
    },
    {
        'id': 'rembg',
        'name': 'Remove Background',
        'url': 'https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg',
    },
    {
        'id': 'regionalPrompter',
        'name': 'Regional Prompter',
        'url': 'https://github.com/hako-mikan/sd-webui-regional-prompter',
    },
    {
        'id': 'sag',
        'name': 'Self Attention Guidance',
        'url': 'https://github.com/ashen-sensored/sd_webui_SAG',
    },
    {
        'id': 'styleXL',
        'name': 'Style Selector XL',
        'url': 'https://github.com/ahgsql/StyleSelectorXL'
    },
    {
        'id': 'tagComplete',
        'name': 'Tag Autocomplete',
        'url': 'https://github.com/DominikDoom/a1111-sd-webui-tagcomplete',
    },
    {
        'id': 'tiledDiffusion',
        'name': 'Tiled Diffusion & VAE',
        'url': 'https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111',
    },
]


#################################### MODELS #####################################
############################# ANIME / CARTOON /3D ###############################

animeModels = [
    {
        'id': 'animagine',
        'name': 'Animagine XL',
        'variants': [
            {
                'version': 'v3',
                'url': 'https://huggingface.co/cagliostrolab/animagine-xl-3.0/resolve/main/animagine-xl-3.0.safetensors'
            }
        ]
    },
    {
        'id': 'bluePencil',
        'name': 'Blue Pencil XL',
        'variants': [
            {
                'version': 'v3.1',
                'url': 'https://huggingface.co/bluepen5805/blue_pencil-XL/resolve/main/blue_pencil-XL-v3.1.0.safetensors'
            }
        ]
    },
    {
        'id': 'counterfeit',
        'name': 'CounterfeitXL',
        'variants': [
            {
                'version': 'v2.5',
                'url': 'https://huggingface.co/gsdf/CounterfeitXL-V2.0/resolve/main/CounterfeitXL-V2.5.safetensors'
            }
        ]
    },
    {
        'id': 'dynaVision',
        'name': 'DynaVision XL',
        'variants': [
            {
                'version': 'v0.6.1.0_vae',
                'url': 'https://civitai.com/api/download/models/297740?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            }
        ]
    },
    {
        'id': 'sdxlNiji',
        'name': 'SDXL Niji',
        'variants': [
            {
                'version': 'SE',
                'url': 'https://huggingface.co/rafationgson/niji-sdxl/resolve/main/sdxlNijiSpecial_sdxlNijiSE.safetensors'
            }
        ]
    },
    {
        'id': 'unstableDiff',
        'name': 'SDXL Unstable Diffusers',
        'variants': [
            {
                'version': 'v11',
                'url': 'https://huggingface.co/frankjoshua/sdxlUnstableDiffusers_v11/resolve/main/sdxlUnstableDiffusers_v11.safetensors'
            },
            {
                'version': 'Turbo v10',
                'url': 'https://civitai.com/api/download/models/247214?type=Model&format=SafeTensor&size=full&fp=fp16'
            }
        ]
    }
]

############################### GENERAL PURPOSE #################################

generalModels = [
    {
        'id': 'copaxTimeless',
        'name': 'Copax TimeLessXL',
        'variants': [
            {
                'version': 'v9',
                'url': 'https://civitai.com/api/download/models/293413?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            },
            {
                'version': 'Turbo v1',
                'url': 'https://civitai.com/api/download/models/247259?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            }
        ]
    },
    {
        'id': 'dreamShaper',
        'name': 'DreamShaper XL',
        'variants': [
            {
                'version': 'aplha2',
                'url': 'https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaperXL1.0Alpha2_fixedVae_half_00001_.safetensors'
            },
            {
                'version': 'Turbo DPM++ SDE',
                'url': 'https://huggingface.co/Lykon/dreamshaper-xl-turbo/resolve/main/DreamShaperXL_Turbo_dpmppSdeKarras_half_pruned_6.safetensors'
            }
        ]
    },
    {
        'id': 'juggernaut',
        'name': 'Juggernaut XL',
        'variants': [
            {
                'version': 'v8',
                'url': 'https://huggingface.co/Dremmar/juggernaut_v8/resolve/main/juggernautXL_v8Rundiffusion.safetensors'
            }
        ]
    },
    {
        'id': 'protoVision',
        'name': 'ProtoVision XL',
        'variants': [
            {
                'version': 'v6.6.0_vae',
                'url': 'https://civitai.com/api/download/models/265938?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            }
        ]
    },
    {
        'id': 'sdxl',
        'name': 'SD XL',
        'variants': [
            {
                'version': 'v1_vae',
                'url': 'https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors'
            },
            {
                'version': 'v1_vae-refiner',
                'url': 'https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors'
            },
            {
                'version': 'Turbo v1',
                'url': 'https://huggingface.co/stabilityai/sdxl-turbo/resolve/main/sd_xl_turbo_1.0_fp16.safetensors'
            }
        ]
    }
]

################################## REALISTIC ####################################

realisticModels = [
    {
        'id': 'newReality',
        'name': 'NewRealityXL',
        'variants': [
            {
                'version': 'v2',
                'url': 'https://civitai.com/api/download/models/275491?type=Model&format=SafeTensor&size=full&fp=fp16'
            }
        ]
    },
    {
        'id': 'nightVision',
        'name': 'NightVision XL',
        'variants': [
            {
                'version': 'v0.7.9.1_vae',
                'url': 'https://civitai.com/api/download/models/261539?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            }
        ]
    },
    {
        'id': 'realismEngine',
        'name': 'Realism Engine SDXL',
        'variants': [
            {
                'version': 'v3_vae',
                'url': 'https://huggingface.co/misri/realismEngineSDXL_v30VAE/resolve/main/realismEngineSDXL_v30VAE.safetensors'
            }
        ]
    },
    {
        'id': 'realEdge',
        'name': 'Realities Edge XL',
        'variants': [
            {
                'version': 'v5',
                'url': 'https://civitai.com/api/download/models/183894?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            },
            {
                'version': 'Turbo v2',
                'url': 'https://civitai.com/api/download/models/294995?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            }
        ]
    },
    {
        'id': 'realVis',
        'name': 'RealVisXL',
        'variants': [
            {
                'version': 'v3_vae',
                'url': 'https://huggingface.co/SG161222/RealVisXL_V3.0/resolve/main/RealVisXL_V3.0.safetensor'
            },
            {
                'version': 'v3_vae-inpainting',
                'url': 'https://civitai.com/api/download/models/297320?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            },
            {
                'version': 'Turbo v3_vae',
                'url': 'https://huggingface.co/SG161222/RealVisXL_V3.0_Turbo/resolve/main/RealVisXL_V3.0_Turbo.safetensors'
            }
        ]
    }
]

#################################### WOMAN #####################################

womanModels = [
    {
        'id': 'arienmix',
        'name': 'ArienMixXL Asian portrait',
        'variants': [
          {
            'version': 'v3',
            'url': 'https://civitai.com/api/download/models/175141?type=Model&format=SafeTensor&size=pruned&fp=bf16'
          }
        ]
    },
    {
        'id': 'helloWorld',
        'name': 'LEOSAM\'s HelloWorld SDXL',
        'variants': [
            {
                'version': 'v3.2',
                'url': 'https://civitai.com/api/download/models/286441?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            },
            {
                'version': 'Turbo+LCM v2',
                'url': 'https://civitai.com/api/download/models/243290?type=Model&format=SafeTensor&size=full&fp=fp16'
            }
        ]
    },
    
    {
        'id': 'sdvnReal',
        'name': 'SDVN6 RealXL',
        'variants': [
          {
            'version': 'Detailface',
            'url': 'https://civitai.com/api/download/models/134461?type=Model&format=SafeTensor&size=full&fp=fp16'
          }
        ]
    },
    {
        'id': 'xxmix9real',
        'name': 'XXMix 9realistic SDXL',
        'variants': [
          {
            'version': 'v1',
            'url': 'https://huggingface.co/Remilistrasza/XXMix_9realisticSDXL/resolve/main/xxmix9realisticsdxl_v10.safetensors'
          }
        ]
    }
]


########################################################################################################
############################################## LORA ####################################################

defaultLoraList = [
    {
        'name': 'Detail Tweaker XL',
        'url': 'https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/detail_tweaker_xl.safetensors',
    }
]

loraList = [
    {
        'id': 'blurXL',
        'name': 'Blur SDXL',
        'url': ['https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/blur_sdxl.safetensors'],
    },
    {
        'id': 'detailEyes',
        'name': 'DetailedEyes XL',
        'url': ['https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/detailedEyes_v3.safetensors'],
    },
    {
        'id': 'lcmXL',
        'name': 'LCM Lora SDXL',
        'url': ['https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/xl_more_art-full_v1.safetensors'],
    },
    {
        'id': 'lcmTurbo',
        'name': 'LCM&TurboMix',
        'url': [
            'https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/LCMTurboMix_DPM_SDE_Karras.safetensors',
            'https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/LCMTurboMix_Euler_A_fix.safetensors',
            'https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/LCMTurboMix_LCM_Sampler.safetensors'
        ]
    },
    {
        'id': 'handsXL',
        'name': 'Hands XL',
        'url': ['https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/hands_xl_v21.safetensors']
    },
    {
        'id': 'moreArt',
        'name': 'XL More Art Full',
        'url': ['https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/xl_more_art-full_v1.safetensors']
    }
    
]


################################### EMBEDDINGS (TEXTUAL INVERSION) ####################################

defaultEmbeddingList = [
    {
        'name': 'Negative prompts pack XL',
        'url': 'https://huggingface.co/ffxvs/negative-prompts-pack-xl',
    }
]

############################################## UPSCALER ##############################################

defaultUpscalerList = [
    {
        'name': 'UltraSharp',
        'url': 'https://huggingface.co/ffxvs/upscaler/resolve/main/UltraSharp_4x.pth',
    },
]

upscalerList = [
    {
        'id': 'bsrgan',
        'name': 'BSRGAN',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/BSRGAN_4x.pth'],
    },
    {
        'id': 'lsdirPlus',
        'name': 'LSDIRPlus',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/LSDIRplus_4x.pth'],
    },
    {
        'id': 'nomos8ksc',
        'name': 'Nomos8kSC',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/Nomos8kSC_4x.pth'],
    },
    {
        'id': 'superscale',
        'name': 'NMKD Superscale',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/NMKD_Superscale_4x.pth'],
    },
    {
        'id': 'remacri',
        'name': 'Remacri',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/Remacri_4x.pth'],
    },
    {
        'id': 'ultraMix',
        'name': 'UltraMix',
        'url': [
            'https://huggingface.co/ffxvs/upscaler/resolve/main/UltraMix_Balanced_4x.pth',
            'https://huggingface.co/ffxvs/upscaler/resolve/main/UltraMix_Restore_4x.pth',
            'https://huggingface.co/ffxvs/upscaler/resolve/main/UltraMix_Smooth_4x.pth'
        ],
    },
    {
        'id': 'valar',
        'name': 'Valar',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/Valar_4x.pth'],
    },
]


############################################## VAE ##############################################

defaultVaeList = [
    {
        'name': 'sdxl-vae-fp16-fix',
        'url': 'https://huggingface.co/ffxvs/vae-collection-xl/resolve/main/sdxl-vae-fp16-fix.safetensors',
    }
]

inf('\u2714 Completed','success', '30px')

### 1.2. Dependencies
**REQUIRED EVERY RUNTIME START**

In [ ]:
print('⏳ Installing dependencies...')
print('This may take a bit of time...')
with capture.capture_output() as cap:
    !pip install -q pickleshare
    %cd {root}
    mallocPath = f'{dependenciesPath}/libtcmalloc_minimal.so.4'
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    os.environ['PYTHONWARNINGS'] = 'ignore'
    !apt-get -y -q update
    !apt-get -y -q install aria2 git-lfs
    !mkdir -p {dependenciesPath}
    
    if not os.path.exists(mallocPath):
        downloader('https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4', dependenciesPath)
    %env LD_PRELOAD = {mallocPath}

    !pip install -q torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
    !pip install -q xformers==0.0.20 triton==2.0.0 ngrok insightface

print('\n')
inf('\u2714 Completed','success', '30px')

# 2. Setup Webui

### 2.1. Webui

In [ ]:
# Current Webui version
webuiVersion = 'v1.7.0'

%cd {root}
print('⏳ Installing Stable Diffusion Webui...')
silentClone(f'-b {webuiVersion} https://github.com/AUTOMATIC1111/stable-diffusion-webui', root)
%cd {webui}
!git remote set-branches origin 'master' && git fetch -q --depth=5 && git checkout -q -f {webuiVersion}

!mkdir -p {modelsPath}/sdxl
!mkdir -p {embeddingsPath}/sdxl
!mkdir -p {loraPath}/sdxl
!mkdir -p {upscalerPath}
!mkdir -p {vaePath}/sdxl

downloader('https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/extras.py', modulesPath, True)
downloader('https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/sd_models.py', modulesPath, True)
downloader('https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/shared_options.py', modulesPath, True)

# From ThelastBen
!sed -i 's@shared.opts.data\["sd_model_checkpoint"] = checkpoint_info.title@shared.opts.data\["sd_model_checkpoint"] = checkpoint_info.title;model.half()@' {modulesPath}/sd_models.py
!sed -i "s@map_location='cpu'@map_location='cuda'@" {modulesPath}/extras.py
!sed -i 's@\["sd_model_checkpoint"\]@\["sd_model_checkpoint", "sd_vae", "CLIP_stop_at_last_layers"\]@g' {modulesPath}/shared_options.py

print('\n')
inf('\u2714 Completed','success', '30px')

### 2.2. Extensions

#### Extension List

**Pre-Installed Extensions :** 

* [Aspect Ratio Helper](https://github.com/thomasasfk/sd-webui-aspect-ratio-helper)
* [Canvas Zoom](https://github.com/richrobber2/canvas-zoom)
* [Cleaner](https://github.com/novitalabs/sd-webui-cleaner)
* [Controlnet](https://github.com/Mikubill/sd-webui-controlnet)
* [SD Delete Button](https://github.com/reforget-id/sd_delete_button)
* [SD Model Downloader](https://github.com/Iyashinouta/sd-model-downloader)
* [State](https://github.com/ilian6806/stable-diffusion-webui-state)
* [Ultimate SD Upscale](https://github.com/Coyote-A/ultimate-upscale-for-automatic1111)

In [ ]:
# UPDATE EXTENSIONS
updateExts = boolean[1]

# ControlNet - https://github.com/Mikubill/sd-webui-controlnet
allControlNet = boolean[0] # Download all controlNet models (The total size will be very large)

controlNet = {  # Select models to download 
                # Default models are : depth, ip-adapter, openpose
    'blur': boolean[0],
    'blur_anime': boolean[0],
    'canny': boolean[0],
    'canny_anime': boolean[0],
    'depth': boolean[0],
    'depth_anime': boolean[0],
    'ipadapter': boolean[1],
    'lineart': boolean[0],
    'openpose': boolean[0],
    'openpose_anime': boolean[0],
    'recolor': boolean[0],
    'scribble_anime': boolean[0],
    'sketch': boolean[0],
    'softedge': boolean[0],
}

# Adetailer (After Detailer) - https://github.com/Bing-su/adetailer
adetailer = boolean[1]

# AnimateDiff - https://github.com/continue-revolution/sd-webui-animatediff
animateDiff = boolean[0]

# BMAB - https://github.com/portu-sim/sd-webui-bmab
bmab = boolean[0]

# Depth Map Library - https://github.com/wywywywy/sd-webui-depth-lib
depthLib = boolean[0]

# Hugging Face - https://github.com/camenduru/stable-diffusion-webui-huggingface
huggingFace = boolean[0]

# Infinite Image Browsing - https://github.com/zanllp/sd-webui-infinite-image-browsing
infiniteImg = boolean[1]

# Inpaint Anything - https://github.com/Uminosachi/sd-webui-inpaint-anything
inpaintAny = boolean[0]

# Latent Couple - https://github.com/aria1th/stable-diffusion-webui-two-shot
latentCouple = boolean[0]

# Mini Paint - https://github.com/0Tick/a1111-mini-paint
miniPaint = boolean[1]

# Negative Prompt Weight - https://github.com/muerrilla/stable-diffusion-NPW
npw = boolean[0]

# openOutpaint - https://github.com/zero01101/openOutpaint-webUI-extension
openOutpaint = boolean[0]

# Photopea - https://github.com/yankooliveira/sd-webui-photopea-embed
photopea = boolean[1]

# Prompt History - https://github.com/namkazt/sd-webui-prompt-history
promptHistory = boolean[0]

# Regional Prompter - https://github.com/hako-mikan/sd-webui-regional-prompter
regionalPrompter = boolean[0]

# Remove Background - https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg
rembg = boolean[0]

# Self Attention Guidance - https://github.com/ashen-sensored/sd_webui_SAG
sag = boolean[0]

# Style Selector XL - https://github.com/ahgsql/StyleSelectorXL
styleXL = boolean[0]

# Tag Autocomplete - https://github.com/DominikDoom/a1111-sd-webui-tagcomplete
tagComplete = boolean[1]

# Tiled Diffusion & VAE - https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111
tiledDiffusion = boolean[1]


################################################################################################################


print("⏳ Installing pre-installed extensions...")
for extension in defaultExtensions:
    print(extension['name'] + '...')
    silentClone(extension['url'], extensionsPath, updateExts)

print("\n⏳ Installing selected extensions...")
for extension in extensionList:
    if eval(extension['id']):
        print(extension['name'] + '...')
        silentClone(extension['url'], extensionsPath, updateExts)

if allControlNet:
    print("\n⏳ Download all acontrolNet models...")
    for model in controlNetURLs:
        print('\n' + model + '...')
        downloader(controlNetURLs[model][0], controlNetModelsPath)
else:
    print("\n⏳ Download selected controlNet models...")
    for model in controlNet:
        if controlNet[model]:
            print('\n' + model + '...')
            downloader(controlNetURLs[model][0], controlNetModelsPath)        

print('\n')        
%cd {webui}
print('\n')
inf('\u2714 Completed','success', '30px')

#### Install from URL

In [ ]:
# UPDATE EXTENSIONS
updateExts = boolean[0]

# Install extensions from URL
otherExtensions = []

if otherExtensions:
    print("⏳ Installing extensions...")  
    for extension in otherExtensions:
        name = extension.split('/')[-1]
        print(name + '...')
        silentClone(extension, extensionsPath, updateExts)

print('\n')        
%cd {webui}
print('\n')
inf('\u2714 Completed','success', '30px')

# 3. Models
**Choose models you want to download**

### Anime / Cartoon / 3D

In [ ]:
# Animagine XL - https://civitai.com/models/260267
animagineVersions = ['Select version...', 'v3']
animagine = animagineVersions[0]

# Blue Pencil XL - https://civitai.com/models/119012
bluePencilVersions = ['Select version...', 'v3.1']
bluePencil = bluePencilVersions[0]

# CounterfeitXL - https://civitai.com/models/118406
counterfeitVersions = ['Select version...', 'v2.5']
counterfeit = counterfeitVersions[0]

# DynaVision XL - https://civitai.com/models/122606
dynaVisionVersions = ['Select version...', 'v0.6.1.0_vae']
dynaVision = dynaVisionVersions[0]

# SDXL Niji - https://civitai.com/models/120765
sdxlNijiVersions = ['Select version...', 'SE']
sdxlNiji = sdxlNijiVersions[0]

# SDXL Unstable Diffusers - https://civitai.com/models/84040
unstableDiffVersions = ['Select version...', 'v11', 'Turbo v10']
unstableDiff = unstableDiffVersions[0]


########################################################################################


print('⏳ Downloading selected models...')
for model in selectedModels(animeModels):
    print('\n* ' + model['name'] + ' | ' + model['version'])
    downloader(model['url'], f'{modelsPath}/sdxl')

print('\n') 
inf('\u2714 Completed','success', '30px')

### General Purpose

In [ ]:
# Copax TimeLessXL - https://civitai.com/models/118111
copaxTimelessVersions = ['Select version...', 'v9', 'Turbo v1']
copaxTimeless = copaxTimelessVersions[0]

# DreamShaper XL - https://civitai.com/models/112902
dreamShaperVersions = ['Select version...', 'aplha2', 'Turbo DPM++ SDE']
dreamShaper = dreamShaperVersions[0]

# Juggernaut XL - https://civitai.com/models/133005
juggernautVersions = ['Select version...', 'v8']
juggernaut = juggernautVersions[0]

# ProtoVision XL - https://civitai.com/models/125703
protoVisionVersions = ['Select version...', 'v6.6.0_vae']
protoVision = protoVisionVersions[0]

# SD XL - https://civitai.com/models/101055
sdxlVersions = ['Select version...', 'v1_vae', 'v1_vae-refiner', 'Turbo v1']
sdxl = sdxlVersions[0]


########################################################################################


print('⏳ Downloading selected models...')
for model in selectedModels(generalModels):
    print('\n* ' + model['name'] + ' | ' + model['version'])
    downloader(model['url'], f'{modelsPath}/sdxl')

print('\n') 
inf('\u2714 Completed','success', '30px')

### Realistic

In [ ]:
# NewRealityXL - https://civitai.com/models/161068
newRealityVersions = ['Select version...', 'v2']
newReality = newRealityVersions[0]

# NightVision XL - https://civitai.com/models/128607
nightVisionVersions = ['Select version...', 'v0.7.9.1_vae']
nightVision = nightVisionVersions[0]

# Realism Engine SDXL - https://civitai.com/models/152525
realismEngineVersion = ['Select version...', 'v3_vae']
realismEngine = realismEngineVersion[0]

# Realities Edge XL - https://civitai.com/models/129666
realEdgeVersions = ['Select version...', 'v5', 'Turbo v2']
realEdge = realEdgeVersions[0]

# RealVisXL - https://civitai.com/models/139562
realVisVersions = ['Select version...', 'v3_vae', 'v3_vae-inpainting', 'Turbo v3_vae']
realVis = realVisVersions[0]


########################################################################################


print('⏳ Downloading selected models...')
for model in selectedModels(realisticModels):
    print('\n* ' + model['name'] + ' | ' + model['version'])
    downloader(model['url'], f'{modelsPath}/sdxl')

print('\n') 
inf('\u2714 Completed','success', '30px')

### Woman

In [ ]:
# ArienMixXL Asian portrait - https://civitai.com/models/118913
arienmixVersions = ['Select version...', 'v3']
arienmix = arienmixVersions[1]

# LEOSAM's HelloWorld SDXL - https://civitai.com/models/43977
helloWorldVersions = ['Select version...', 'v3.2', 'Turbo+LCM v2']
helloWorld = helloWorldVersions[0]

# SDVN6 RealXL - https://civitai.com/models/118114
sdvnRealVersions = ['Select version...', 'Detailface']
sdvnReal = sdvnRealVersions[0]

# XXMix 9realistic SDXL - https://civitai.com/models/124421
xxmix9realVersions = ['Select version...', 'v1']
xxmix9real = xxmix9realVersions[0]


########################################################################################


print('⏳ Downloading selected models...')
for model in selectedModels(womanModels):
    print('\n* ' + model['name'] + ' | ' + model['version'])
    downloader(model['url'], f'{modelsPath}/sdxl')

print('\n') 
inf('\u2714 Completed','success', '30px')

### Install from URL

In [ ]:
# Install models from URL 
otherModels = []


########################################################################################


if otherModels:
    print('⏳ Downloading models...')
    for model in otherModels:
        print('\n* ' + model)
        downloader(model, f'{modelsPath}/sdxl')
        
print('\n')
%cd {webui}
print('\n')
inf('\u2714 Completed','success', '30px')

# 4. LoRA, Embedding, Upscaler and VAE

### Resources List

**Pre-installed LoRA :**
* [Detail Tweaker XL](https://civitai.com/models/122359)

**Pre-installed Embeddings :**
* [Negative Prompts Pack XL](https://huggingface.co/ffxvs/negative-prompts-pack-xl)

**Pre-installed Upscaler :**
* [UltraSharp](https://openmodeldb.info/models/4x-UltraSharp)

**Pre-installed VAE :**
* [sdxl-vae-fp16-fix](https://huggingface.co/madebyollin/sdxl-vae-fp16-fix)

In [ ]:
####################################### LORA ##########################################

# Blur SDXL - https://civitai.com/models/124855
blurXL = boolean[0]

# DetailedEyes XL - https://civitai.com/models/120723
detailEyes = boolean[0]

# LCM Lora SDXL - https://huggingface.co/latent-consistency/lcm-lora-sdxl
lcmXL = boolean[1]

# LCM&TurboMix - https://civitai.com/models/216190
lcmTurbo = boolean[0]

# Hands XL - https://civitai.com/models/200255
handsXL = boolean[0]

# XL More Art Full - https://civitai.com/models/124347
moreArt = boolean[0]


###################################### UPSCALER ########################################

# BSRGAN - https://openmodeldb.info/models/4x-BSRGAN
bsrgan = boolean[0]

# LSDIRPlus - https://openmodeldb.info/models/4x-LSDIRplus
lsdirPlus = boolean[1]

# NMKD Superscale - https://openmodeldb.info/models/4x-NMKD-Superscale
superscale = boolean[0]

# Nomos8kSC - https://openmodeldb.info/models/4x-Nomos8kSC
nomos8ksc = boolean[0]

# Remacri - https://openmodeldb.info/models/4x-Remacri
remacri = boolean[0]

# UltraMix - https://upscale.wiki/w/index.php?title=Model_Database&oldid=1571
ultraMix = boolean[0]

# Valar - https://openmodeldb.info/models/4x-Valar
valar = boolean[0]


###################################################################################################

# LoRA
def downloadLora():
    print('⏳ Downloading pre-installed LoRA...')
    defaultResources(defaultLoraList, f'{loraPath}/sdxl')
    print('\n\n⏳ Downloading selected LoRA...')
    selectedResources(loraList, f'{loraPath}/sdxl')    

# Embeddings
def downloadEmbeddings():
    print('\n\n⏳ Downloading pre-installed embeddings...')
    for embedding in defaultEmbeddingList:
        print(embedding['name'] + '...')
        silentClone(embedding['url'], f'{embeddingsPath}/sdxl', True)

# Upscaler
def downloadUpscaler():
    print('\n\n⏳ Downloading pre-installed upscaler...')
    defaultResources(defaultUpscalerList, upscalerPath)
    print('\n\n⏳ Downloading selected upscaler...')
    selectedResources(upscalerList, upscalerPath)

# VAE 
def downloadVAE():
    print('\n\n⏳ Downloading pre-installed VAE...')
    defaultResources(defaultVaeList, f'{vaePath}/sdxl')

%cd {webui}
downloadLora()
downloadEmbeddings()
downloadUpscaler()
downloadVAE()

print('\n')
%cd {webui}
print('\n')
inf('\u2714 Completed','success', '30px')

### Install from URL

In [ ]:
# LoRA
otherLora = []

# Embeddings
otherEmbeddings = []

# Upscaler
otherUpscaler = []

# VAE
otherVAE = []


###################################################################################################

if otherLora:
    print('\n\n⏳ Downloading LoRA...')
    otherResources(otherLora, f'{loraPath}/sdxl')
if otherEmbeddings:
    print('\n\n⏳ Downloading embeddings...')
    otherResources(otherEmbeddings, f'{embeddingsPath}/sdxl')
if otherUpscaler:
    print('\n\n⏳ Downloading upscaler...')
    otherResources(otherUpscaler, upscalerPath)
if otherVAE:
    print('\n\n⏳ Downloading VAE...')
    otherResources(otherVAE, f'{vaePath}/sdxl')

print('\n')
%cd {webui}
print('\n')
inf('\u2714 Completed','success', '30px')

# 5. Launch Webui

Wait until `Model loaded in **.*s ...` appears.  
Then, click Gradio link `https://xxxxx.gradio.live` or Ngrok link `https://xxxxx.ngrok-free.app` to open WebUI.  
Always watch the output log to track image generation progress or errors. 

In [ ]:
boolean = [False, True]

# Dark theme
darkTheme = boolean[1]

# Optimizations
# For "opt-sdp-attention" and "opt-sdp-no-mem-attention" set directly from webui settings
xformers = boolean[1]
medvram = boolean[0]

# Ngrok (Optional)
ngrokToken = ''
ngrokDomain = ''

# CORS (Optional) 
# separate with commas
cors = 'https://huchenlei.github.io'


################################################################################################################

import json 
%cd {webui}
print('⏳ Preparing...')
print('It will take a little longer for the first time...')
args = '--disable-safe-unpickle --enable-insecure-extension-access --gradio-queue --api --listen'

if darkTheme:
    args += ' --theme dark'
if xformers:
    args += ' --xformers'
if medvram:
    args += ' --medvram'
if ngrokToken:
    args += ' --ngrok ' + ngrokToken
    if ngrokDomain:
        ngrokOptions = '\'{"hostname":"' + ngrokDomain + '", "request_header.add":"ngrok-skip-browser-warning:skip"}\''
        args += ' --ngrok-options ' + ngrokOptions
else:
    args += ' --share'
if cors:
    args += ' --cors-allow-origins ' + cors

with capture.capture_output() as cap:
    !python launch.py --exit 
    !pip install -q pillow==9.5.0

print('Launching Webui...')
!python webui.py {args}

# 6. Zip Outputs
After the output folder is zipped, you can download it from file browser in the left panel.  
It's in `stable-diffusion-webui` folder. Just right click to download.

In [ ]:
%cd {webui}
print('⏳ Compressing files into outputs.zip...')
!zip -r -q outputs.zip outputs
print('The file is ready in /stable-diffusion-webui/outputs.zip')

inf('\u2714 Completed','success', '30px')